In [1]:
%pip install python-dotenv


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import json

from dotenv import load_dotenv
import os

load_dotenv()

cloudflareAccID = os.getenv("CLOUDFLARE_ACC_ID")
cloudflareToken = os.getenv("CLOUDFLARE_TOKEN")

In [3]:
cloudflareAccID

'35ac6816b7cdf6a0602aa8f92c28c127'

In [4]:
cloudflareToken

'k0tsYOcg_-9RkbMTFYEjO4Y7UkdYGxNLhUWYYImh'

In [9]:
def isValidJson(myjson):
  try:
    json.loads(myjson)
  except ValueError as e:
    return False
  return True

In [10]:
def get_social_score(text):
  model = '@cf/meta/llama-2-7b-chat-int8'
  # Define the API endpoint
  url = f"https://api.cloudflare.com/client/v4/accounts/{cloudflareAccID}/ai/run/" + model
  systemPrompt = """
  I will tips you $200 if you satisfy my requirements.
  You are a stock professional, your job is to classify a paragraph to be related any stock in Nasdaq Stock Market, I will give you a paragraph. Classify the paragraph that it is related to stock in Nasdaq (output null if no stock related), give it a score from -1 to 1 (positive (1) / neutral (0) / negative (-1)), do not explain your answer and your answer must in format json { "stockID": `stock ticker`, "score": 'score'} } You must not say anything except the json even you cannot classify.
  """
  # Define the data you want to send in the POST request
  data = {
    "max_tokens": 20,
    "messages": [
      { "role": "system", "content": systemPrompt },
      { "role": "user", "content": text },
    ],
  }

  # Convert the data to JSON format
  data_json = json.dumps(data)

  # Set the headers for the request
  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {cloudflareToken}'
  }

  # Make the POST request
  response = requests.post(url, data=data_json, headers=headers)

  if not isValidJson(response.text):
    return (None, 0)

  response_json = json.loads(response.text)

  if not isValidJson(response_json['result']['response']):
    return (None, 0)
  
  resultJson = json.loads(response_json['result']['response'])
  
  result = resultJson['stockID'], resultJson['score']

  return (result[0], result[1])


In [11]:
import pandas as pd

posts = pd.read_csv('./data/tweets/tweets_clean.csv')
posts

,Date,Text
0,1.517098e+09,MORE than 400 companies report earnings next w...
1,1.517875e+09,Good traders rely on charts. Great traders rel...
2,1.517011e+09,Notable Earnings 📈📉\n\nM\n$IDTI $LMT $RMBS\n\n...
3,1.517011e+09,"Some implied moves for next week earnings, it ..."
4,1.517184e+09,"Can't say for certain obviously, but all these..."
5,1.517443e+09,Yo get ready. Straight electricity is on the w...
6,1.517011e+09,Expect the following to happen in 2018:\n\n1. ...
7,1.517270e+09,lot of speculation when to buy $AAPL. I want ...
8,1.517443e+09,Traders on earnings day.... \n$AMZN +$50 !!\n$...
9,1.517098e+09,growth % in past 3 months:\n\n$NFLX 38.43%\n$...


In [12]:
# Generate result using pandas
stockIDs = []
scores = []
for value in posts["Text"]:
    stockID, score = get_social_score(value)
    stockIDs.append(stockID)
    scores.append(score)

In [13]:
posts["Ticker"] = stockIDs
posts["Score"] = scores
posts

,Date,Text,Ticker,Score
0,1.517098e+09,MORE than 400 companies report earnings next w...,LMT,1
1,1.517875e+09,Good traders rely on charts. Great traders rel...,AMZN,1
2,1.517011e+09,Notable Earnings 📈📉\n\nM\n$IDTI $LMT $RMBS\n\n...,None,0
3,1.517011e+09,"Some implied moves for next week earnings, it ...",None,0
4,1.517184e+09,"Can't say for certain obviously, but all these...",AAPL,-1
5,1.517443e+09,Yo get ready. Straight electricity is on the w...,None,0
6,1.517011e+09,Expect the following to happen in 2018:\n\n1. ...,None,0
7,1.517270e+09,lot of speculation when to buy $AAPL. I want ...,None,0
8,1.517443e+09,Traders on earnings day.... \n$AMZN +$50 !!\n$...,AAPL,1
9,1.517098e+09,growth % in past 3 months:\n\n$NFLX 38.43%\n$...,None,0


In [14]:
posts.to_csv('data\\tweets\\tweets_scored.csv', index=False)